Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
start = time.time()

In [2]:
import pandas as pd, numpy as np, gc
from datetime import datetime
import joblib

import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
#import cudf, cupy
#cudf.__version__

In [3]:
from numba import jit, njit, prange
from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(gt, pred, nafill=True):
    if nafill:
        pred[ np.isnan(pred) ] = np.nanmean( pred )
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

@njit
def numba_log_loss(y,x):
    n = x.shape[0]
    ll = 0.
    for i in prange(n):
        if y[i]<=0.:
            ll += np.log(1-x[i] + 1e-15 )
        else:
            ll += np.log(x[i] + 1e-15)
    return -ll / n

def compute_rce(gt , pred, nafill=True, verbose=0):
    if nafill:
        pred[ np.isnan(pred) ] = np.nanmean( pred )
        
    cross_entropy = numba_log_loss( gt, pred  )
    
    yt = np.mean(gt>0)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    
    if verbose:
        print( "logloss: {0:.5f} / {1:.5f} = {2:.5f}".format(cross_entropy, strawman_cross_entropy, cross_entropy/strawman_cross_entropy))
        print( 'mean:    {0:.5f} / {1:.5f}'.format( np.nanmean( pred ) , yt  ) )
    
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0


In [4]:
def save_memory( df ):
    features = df.columns
    for i in range( df.shape[1] ):
        if df.dtypes[i] == 'uint8':
            df[features[i]] = df[features[i]].astype( np.int8 )
            gc.collect()
        elif df.dtypes[i] == 'bool':
            df[features[i]] = df[features[i]].astype( np.int8 )
            gc.collect()
        elif df.dtypes[i] == 'uint32':
            df[features[i]] = df[features[i]].astype( np.int32 )
            gc.collect()
        elif df.dtypes[i] == 'int64':
            df[features[i]] = df[features[i]].astype( np.int32 )
            gc.collect()
        elif df.dtypes[i] == 'float64':
            df[features[i]] = df[features[i]].astype( np.float32 )
            gc.collect()

# Load Train

In [5]:
train = pd.read_parquet( './dask_input/step2_output/train.parquet' )
test0 = pd.read_parquet( './dask_input/step2_output/valid.parquet' )
test1 = pd.read_parquet( './dask_input/step2_output/test.parquet' )
gc.collect()

train['tr'] = 0
test0['tr'] = 1
test1['tr'] = 1

train.shape, test0.shape, test1.shape

((21043825, 28), (0, 28), (0, 28))

In [6]:
train = pd.concat( (train,test0,test1), sort=False )
gc.collect()
del test0,test1
gc.collect()

train.shape

(21043825, 28)

In [7]:
%%time
train = train.sort_values('timestamp').reset_index(drop=True) #TIME ORDER
gc.collect()

CPU times: user 12.9 s, sys: 2.03 s, total: 14.9 s
Wall time: 14.9 s


20

In [8]:
train.loc[ train.reply>0, 'reply' ] = 1
train.loc[ train.retweet>0, 'retweet' ] = 1
train.loc[ train.retweet_comment>0, 'retweet_comment' ] = 1
train.loc[ train.like>0, 'like' ] = 1

train['engage'] = 0
train.loc[ (train.reply>0)|(train.retweet>0)|(train.retweet_comment>0)|(train.like>0)  , 'engage'] = 1
gc.collect()

19

In [9]:
train.head()

,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,domains,links,hashtags,tr,engage
0,1580947200,27447,115,True,1520574998,0,36,False,1567781844,False,0,0,0,0,326597,11,2,12,11027338,7364305,8110941,2,1,1,75539,885711,105355,0,0
1,1580947200,13782,327,True,1306013762,29,229,False,1411508008,False,0,0,0,0,634520,54,2,4,8294199,3882491,10148536,2,0,0,0,0,753162,0,0
2,1580947200,67167,2864,True,1214456422,114,218,False,1415584664,False,0,0,0,0,1257148,54,2,12,2463142,8702908,2764714,1,0,0,0,0,675884,0,0
3,1580947200,1923062,0,False,1400281110,217,159,False,1498869581,False,0,0,0,0,1161714,59,2,4,12817572,672903,9486466,0,0,0,0,0,0,0,0
4,1580947200,2982406,2411,True,1179400159,41,137,False,1272241451,False,0,0,0,0,802517,54,2,12,9833989,1170946,8165268,2,0,0,0,0,915393,0,0


In [10]:
save_memory(train)
gc.collect()

0

In [11]:
train.dtypes

timestamp             int32
a_follower_count      int32
a_following_count     int32
a_is_verified          int8
a_account_creation    int32
b_follower_count      int32
b_following_count     int32
b_is_verified          int8
b_account_creation    int32
b_follows_a            int8
reply                 int32
retweet               int32
retweet_comment       int32
like                  int32
id                    int32
language               int8
tweet_type             int8
media                  int8
tweet_id              int32
a_user_id             int32
b_user_id             int32
len_hashtags          int32
len_domains           int32
len_links             int32
domains               int32
links                 int32
hashtags              int32
tr                    int32
engage                int32
dtype: object

In [12]:
dt0 = train[['tr','a_user_id','a_follower_count','a_following_count','timestamp','id']].copy()
dt1 = train[['tr','b_user_id','b_follower_count','b_following_count','timestamp','id']].copy()
dt1.columns = ['tr','a_user_id','a_follower_count','a_following_count','timestamp','id']
dt1['id'] = -1
dt = pd.concat( (dt0,dt1), sort=False )
del dt0,dt1; _=gc.collect()
dt.head()

,tr,a_user_id,a_follower_count,a_following_count,timestamp,id
0,0,7364305,27447,115,1580947200,326597
1,0,3882491,13782,327,1580947200,634520
2,0,8702908,67167,2864,1580947200,1257148
3,0,672903,1923062,0,1580947200,1161714
4,0,1170946,2982406,2411,1580947200,802517


In [13]:
%%time
dt['a_fc_max'] = dt.groupby(['tr','a_user_id','a_follower_count'])['timestamp'].transform('max'); _=gc.collect()
dt['a_fc_min'] = dt.groupby(['tr','a_user_id','a_follower_count'])['timestamp'].transform('min'); _=gc.collect()

dt['a_fc_dif'] = dt['a_fc_max'] - dt['a_fc_min']; _=gc.collect()

dt['a_fc_dif_max'] = dt.groupby(['tr','a_user_id'])['a_fc_dif'].transform('max'); _=gc.collect()

dt['a_fc_dif_flag'] = 1* ((dt['a_fc_dif'] == dt['a_fc_dif_max']) )  ; _=gc.collect()
dt.loc[ dt.a_fc_dif==0 ,'a_fc_dif_flag'] = -1

del dt['a_fc_max'],dt['a_fc_min'],dt['a_fc_dif'],dt['a_fc_dif_max'] ; _=gc.collect()
dt['a_fc_dif_flag'].value_counts()

CPU times: user 1min 9s, sys: 9.24 s, total: 1min 18s
Wall time: 1min 18s


 1    23475034
-1    10280925
 0     8331691
Name: a_fc_dif_flag, dtype: int64

In [14]:
train['a_user_fer_count_delta_time'] = train.merge( dt, on='id', how='left' )['a_fc_dif_flag'] ; _=gc.collect()

del dt['a_fc_dif_flag'] ; _=gc.collect()

train['a_user_fer_count_delta_time'].value_counts()

 1    9556576
 0    6823322
-1    4663927
Name: a_user_fer_count_delta_time, dtype: int64

In [15]:
train.groupby(['tr','a_user_fer_count_delta_time'])[['reply','retweet','retweet_comment','like']].agg(['mean'])

reply   retweet retweet_comment      like
                                    mean      mean            mean      mean
tr a_user_fer_count_delta_time                                              
0  -1                           0.004260  0.017557        0.001196  0.067782
    0                           0.001753  0.007218        0.000492  0.027965
    1                           0.000587  0.002356        0.000168  0.009200

In [16]:
%%time
dt['a_fc_max'] = dt.groupby(['tr','a_user_id','a_following_count'])['timestamp'].transform('max'); _=gc.collect()
dt['a_fc_min'] = dt.groupby(['tr','a_user_id','a_following_count'])['timestamp'].transform('min'); _=gc.collect()

dt['a_fc_dif'] = dt['a_fc_max'] - dt['a_fc_min']; _=gc.collect()

dt['a_fc_dif_max'] = dt.groupby(['tr','a_user_id'])['a_fc_dif'].transform('max'); _=gc.collect()

dt['a_fc_dif_flag'] = 1* ((dt['a_fc_dif'] == dt['a_fc_dif_max']) )  ; _=gc.collect()
dt.loc[ dt.a_fc_dif==0 ,'a_fc_dif_flag'] = -1

del dt['a_fc_max'],dt['a_fc_min'],dt['a_fc_dif'],dt['a_fc_dif_max'] ; _=gc.collect()
dt['a_fc_dif_flag'].value_counts()

CPU times: user 1min 9s, sys: 9.16 s, total: 1min 18s
Wall time: 1min 18s


 1    25609828
-1     9855889
 0     6621933
Name: a_fc_dif_flag, dtype: int64

In [17]:
train['a_user_fing_count_delta_time'] = train.merge( dt, on='id', how='left' )['a_fc_dif_flag'] ; _=gc.collect()

del dt['a_fc_dif_flag'] ; _=gc.collect()

train['a_user_fing_count_delta_time'].value_counts()

 1    12102651
 0     5106834
-1     3834340
Name: a_user_fing_count_delta_time, dtype: int64

In [18]:
train.groupby(['tr','a_user_fing_count_delta_time'])[['reply','retweet','retweet_comment','like']].agg(['mean'])

reply   retweet retweet_comment      like
                                     mean      mean            mean      mean
tr a_user_fing_count_delta_time                                              
0  -1                            0.004147  0.017024        0.001162  0.065931
    0                            0.001411  0.005802        0.000398  0.022400
    1                            0.001184  0.004854        0.000335  0.018811

In [19]:
%%time
dt['a_fc_max'] = dt.groupby(['tr','a_user_id','a_follower_count','a_following_count'])['timestamp'].transform('max'); _=gc.collect()
dt['a_fc_min'] = dt.groupby(['tr','a_user_id','a_follower_count','a_following_count'])['timestamp'].transform('min'); _=gc.collect()

dt['a_fc_dif'] = dt['a_fc_max'] - dt['a_fc_min']; _=gc.collect()

dt['a_fc_dif_max'] = dt.groupby(['tr','a_user_id'])['a_fc_dif'].transform('max'); _=gc.collect()

dt['a_fc_dif_flag'] = 1* ((dt['a_fc_dif'] == dt['a_fc_dif_max']) )  ; _=gc.collect()
dt.loc[ dt.a_fc_dif==0 ,'a_fc_dif_flag'] = -1

del dt['a_fc_max'],dt['a_fc_min'],dt['a_fc_dif'],dt['a_fc_dif_max'] ; _=gc.collect()
dt['a_fc_dif_flag'].value_counts()

CPU times: user 1min 14s, sys: 10.4 s, total: 1min 25s
Wall time: 1min 25s


 1    21280683
-1    12491523
 0     8315444
Name: a_fc_dif_flag, dtype: int64

In [20]:
train['a_user_fering_count_delta_time'] = train.merge( dt, on='id', how='left' )['a_fc_dif_flag'] ; _=gc.collect()

del dt['a_fc_dif_flag'] ; _=gc.collect()

train['a_user_fering_count_delta_time'].value_counts()

 1    9329520
 0    6751724
-1    4962581
Name: a_user_fering_count_delta_time, dtype: int64

In [21]:
train.groupby(['tr','a_user_fering_count_delta_time'])[['reply','retweet','retweet_comment','like']].agg(['mean'])

reply   retweet retweet_comment  \
                                       mean      mean            mean   
tr a_user_fering_count_delta_time                                       
0  -1                              0.004185  0.017206        0.001169   
    0                              0.001763  0.007272        0.000499   
    1                              0.000511  0.002054        0.000147   

                                       like  
                                       mean  
tr a_user_fering_count_delta_time            
0  -1                              0.066478  
    0                              0.028157  
    1                              0.008023

In [22]:
%%time
dt['a_fc_count'] = dt.groupby(['tr','a_user_id','a_following_count'])['id'].transform('count'); _=gc.collect()
dt['a_fc_max']   = dt.groupby(['tr','a_user_id'])['a_fc_count'].transform('max'); _=gc.collect()
dt['a_fc_count_flag'] = 1* ((dt['a_fc_count'] == dt['a_fc_max']))  ; _=gc.collect()
dt.loc[ dt.a_fc_count<=1,'a_fc_count_flag'] = -1

del dt['a_fc_count'], dt['a_fc_max']
dt['a_fc_count_flag'].value_counts()

CPU times: user 42.8 s, sys: 5.35 s, total: 48.1 s
Wall time: 48.1 s


 1    28278130
-1     9534282
 0     4275238
Name: a_fc_count_flag, dtype: int64

In [23]:
train['a_user_fing_count_mode'] = train.merge( dt, on='id', how='left' )['a_fc_count_flag'] ; _=gc.collect()
del dt['a_fc_count_flag'] ; _=gc.collect()
train['a_user_fing_count_mode'].value_counts()

 1    14215985
-1     3468577
 0     3359263
Name: a_user_fing_count_mode, dtype: int64

In [24]:
train.groupby(['tr','a_user_fing_count_mode'])[['reply','retweet','retweet_comment','like']].agg(['mean'])

reply   retweet retweet_comment      like
                               mean      mean            mean      mean
tr a_user_fing_count_mode                                              
0  -1                      0.004346  0.017915        0.001221  0.069329
    0                      0.003245  0.013332        0.000906  0.051663
    1                      0.000806  0.003287        0.000229  0.012721

In [25]:
%%time
dt['a_fc_count'] = dt.groupby(['tr','a_user_id','a_follower_count'])['id'].transform('count'); _=gc.collect()
dt['a_fc_max']   = dt.groupby(['tr','a_user_id'])['a_fc_count'].transform('max'); _=gc.collect()
dt['a_fc_count_flag'] = 1* ((dt['a_fc_count'] == dt['a_fc_max']))  ; _=gc.collect()
dt.loc[ dt.a_fc_count<=1,'a_fc_count_flag'] = -1

del dt['a_fc_count'], dt['a_fc_max']
dt['a_fc_count_flag'].value_counts()

CPU times: user 40.1 s, sys: 5.29 s, total: 45.4 s
Wall time: 45.4 s


 1    25440240
-1     9736316
 0     6911094
Name: a_fc_count_flag, dtype: int64

In [26]:
train['a_user_fer_count_mode'] = train.merge( dt, on='id', how='left' )['a_fc_count_flag'] ; _=gc.collect()
del dt['a_fc_count_flag'] ; _=gc.collect()
train['a_user_fer_count_mode'].value_counts()

 1    11053740
 0     5951835
-1     4038250
Name: a_user_fer_count_mode, dtype: int64

In [27]:
train.groupby(['tr','a_user_fer_count_mode'])[['reply','retweet','retweet_comment','like']].agg(['mean'])

reply   retweet retweet_comment      like
                              mean      mean            mean      mean
tr a_user_fer_count_mode                                              
0  -1                     0.004588  0.018932        0.001285  0.073095
    0                     0.002460  0.010081        0.000689  0.039083
    1                     0.000386  0.001556        0.000113  0.006068

In [28]:
%%time
dt['a_fc_count'] = dt.groupby(['tr','a_user_id','a_follower_count','a_following_count'])['id'].transform('count'); _=gc.collect()
dt['a_fc_max']   = dt.groupby(['tr','a_user_id'])['a_fc_count'].transform('max'); _=gc.collect()
dt['a_fc_count_flag'] = 1* ((dt['a_fc_count'] == dt['a_fc_max']))  ; _=gc.collect()
dt.loc[ dt.a_fc_count<=1,'a_fc_count_flag'] = -1

del dt['a_fc_count'], dt['a_fc_max']
dt['a_fc_count_flag'].value_counts()

CPU times: user 42.3 s, sys: 5.61 s, total: 47.9 s
Wall time: 47.9 s


 1    23294352
-1    11907899
 0     6885399
Name: a_fc_count_flag, dtype: int64

In [29]:
train['a_user_fering_count_mode'] = train.merge( dt, on='id', how='left' )['a_fc_count_flag'] ; _=gc.collect()
del dt['a_fc_count_flag'] ; _=gc.collect()
train['a_user_fering_count_mode'].value_counts()

 1    10789284
 0     5931169
-1     4323372
Name: a_user_fering_count_mode, dtype: int64

In [30]:
train.groupby(['tr','a_user_fering_count_mode'])[['reply','retweet','retweet_comment','like']].agg(['mean'])

reply   retweet retweet_comment      like
                                 mean      mean            mean      mean
tr a_user_fering_count_mode                                              
0  -1                        0.004487  0.018475        0.001249  0.071388
    0                        0.002430  0.009974        0.000685  0.038636
    1                        0.000336  0.001355        0.000099  0.005290

In [31]:
del dt; _=gc.collect()

In [32]:
train['a_count_combined'] = pd.factorize(
    (1+train['a_user_fer_count_delta_time'])*3**0 +
    (1+train['a_user_fing_count_delta_time'])*3**1 +
    (1+train['a_user_fering_count_delta_time'])*3**2 +
    (1+train['a_user_fing_count_mode'])*3**3 +
    (1+train['a_user_fer_count_mode'])*3**4 +
    (1+train['a_user_fering_count_mode'])*3**5 
)[0]
train['a_count_combined'].value_counts()

10     7555000
1      3234213
0      2235152
2      2198100
30     1303397
18      641070
3       604861
28      591446
5       463323
20      436671
27      278323
24      184788
17      133396
4       123534
7       110406
8        81533
47       76629
39       73295
21       66184
12       62598
43       52251
6        50718
33       42632
31       37211
13       31357
23       30550
11       26659
9        25455
53       22891
25       17986
38       16725
60       15646
42       13384
69       12572
16       11689
56       10577
40       10294
15       10207
63       10148
19        7342
35        7115
62        6939
71        6204
54        5737
46        5707
48        5686
59        5598
32        5168
55        5017
34        4676
80        4320
51        3909
70        3520
84        3034
109       2800
78        2674
37        2632
76        2553
75        2470
50        2458
26        2355
114       2198
22        2195
65        2176
61        1979
64        1880
36        

In [33]:
train.loc[ train.tr==0 ].groupby('a_count_combined')[['reply','retweet','retweet_comment','like']].agg(['mean'])

,reply,retweet,retweet_comment,like
,mean,mean,mean,mean
a_count_combined,,,,
0,0.002506,0.010255,0.000721,0.039620
1,0.004430,0.018314,0.001253,0.070833
2,0.001855,0.007607,0.000510,0.029368
3,0.004214,0.017263,0.001179,0.066551
4,0.006306,0.024511,0.001805,0.095852
5,0.004084,0.017355,0.001168,0.068447
6,0.013881,0.055720,0.003667,0.211444
7,0.007853,0.032806,0.001993,0.126325


In [34]:
train.head()

,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,domains,links,hashtags,tr,engage,a_user_fer_count_delta_time,a_user_fing_count_delta_time,a_user_fering_count_delta_time,a_user_fing_count_mode,a_user_fer_count_mode,a_user_fering_count_mode,a_count_combined
0,1580947200,27447,115,1,1520574998,0,36,0,1567781844,0,0,0,0,0,326597,11,2,12,11027338,7364305,8110941,2,1,1,75539,885711,105355,0,0,0,0,0,0,0,0,0
1,1580947200,13782,327,1,1306013762,29,229,0,1411508008,0,0,0,0,0,634520,54,2,4,8294199,3882491,10148536,2,0,0,0,0,753162,0,0,-1,-1,-1,-1,-1,-1,1
2,1580947200,67167,2864,1,1214456422,114,218,0,1415584664,0,0,0,0,0,1257148,54,2,12,2463142,8702908,2764714,1,0,0,0,0,675884,0,0,-1,-1,-1,-1,-1,-1,1
3,1580947200,1923062,0,0,1400281110,217,159,0,1498869581,0,0,0,0,0,1161714,59,2,4,12817572,672903,9486466,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2
4,1580947200,2982406,2411,1,1179400159,41,137,0,1272241451,0,0,0,0,0,802517,54,2,12,9833989,1170946,8165268,2,0,0,0,0,915393,0,0,0,1,0,1,0,0,2


In [35]:
dt = train[['id','a_count_combined','a_user_fer_count_delta_time','a_user_fing_count_delta_time','a_user_fering_count_delta_time','a_user_fing_count_mode','a_user_fer_count_mode','a_user_fering_count_mode']]
dt = dt.sort_values( 'id' )
dt = dt.reset_index( drop=True )
save_memory( dt )
dt.to_parquet( 'a_count_combined-final.parquet' )
dt.tail(5)

,id,a_count_combined,a_user_fer_count_delta_time,a_user_fing_count_delta_time,a_user_fering_count_delta_time,a_user_fing_count_mode,a_user_fer_count_mode,a_user_fering_count_mode
21043775,20872892,2,0,1,0,1,0,0
21043776,20872892,10,1,1,1,1,1,1
21043777,20872893,2,0,1,0,1,0,0
21043778,20872894,10,1,1,1,1,1,1
21043779,20872895,10,1,1,1,1,1,1
21043780,20872896,2,0,1,0,1,0,0
21043781,20872897,10,1,1,1,1,1,1
21043782,20872898,2,0,1,0,1,0,0
21043783,20872899,17,-1,1,-1,1,0,0
21043784,20872900,2,0,1,0,1,0,0
